In [1]:
 # -*- coding: utf-8 -*-
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import pandas as pd
import numpy as np
import japanize_matplotlib
import os

train_file = pd.read_csv(r"C:\Users\Kenichi\Documents\Kaggle_Data\kuzushiji_recognition\train.csv")

unicode_chart = pd.read_csv(r"C:\Users\Kenichi\Documents\Kaggle_Data\kuzushiji_recognition\unicode_translation.csv")
train_file.labels = train_file.labels.str.split(' ') #(3881, 2)
train_file.labels.head(3)

0    [U+306F, 1231, 3465, 133, 53, U+304C, 275, 165...
1    [U+306F, 1087, 2018, 103, 65, U+304B, 1456, 18...
2    [U+306F, 572, 1376, 125, 57, U+306E, 1551, 208...
Name: labels, dtype: object

In [3]:
#changed the train label dimension
label_np = np.array(train_file.labels[0])
label_np = label_np.reshape(int(len(label_np)/5), 5)
label_list = np.array([])
for i in label_np[:, 0]:
    label_list= np.append(label_list, unicode_chart.char[unicode_chart.Unicode == i])
#label_list = pd.DataFrame(label_list, columns=["chart_index", "char"])
print(label_list)


['は' 'が' 'い' 'け' 'に' 'に' 'の' '工' 'こ' '三' 'の' 'や' 'と' '己' 'も' 'さ' 'の' 'の'
 '世' '細' 'そ' '人' 'け' 'れ' 'だ' '強' 'の' 'た' 'て' '俳' '根' 'か' '諧' 'れ' '子' 'を'
 '及' '者' 'ず' '避' '正' '文' 'ふ' '時' '思' 'な' 'め' '浮' 'を' '気' '職' '老' '武' '楽'
 'も' '盲' '若' '自' 'ど' '風' 'に' '裏' '畳' '息' '序' 'し' 'し']


In [ ]:
def get_padded_image(crop):
    h, w = crop.shape
    if w > h:
        crop_resize = cv2.resize(crop , (64, int(h/(w/64))))
    else:
        ratio = int(h/64)
        crop_resize = cv2.resize(crop , (int(w/(h/64)), 64))

    n_h, n_w = crop_resize.shape
    #print(int((64 - n_h)/2),int((64 - n_w)/2))
    
    # polarize
    crop_resize[np.where(crop_resize > 180)] = 255
    crop_resize[np.where(crop_resize <= 180)] = 0

    # Padding to 64 * 64 pixels 
    bg_color = 255  # crop_resize[0, 0]
    if (64 - n_h)%2 == 0 and (64 - n_w)%2 == 0:
        padded_img = cv2.copyMakeBorder(crop_resize,int((64 - n_h)/2), int((64 - n_h)/2), 
                                    int((64 - n_w)/2), int((64 - n_w)/2), cv2.BORDER_CONSTANT, value = bg_color)
    elif (64 - n_h)%2 != 0 and (64 - n_w)%2 == 0:
        padded_img = cv2.copyMakeBorder(crop_resize,int((64 - n_h)/2), int((64 - n_h)/2) + 1, 
                                    int((64 - n_w)/2), int((64 - n_w)/2), cv2.BORDER_CONSTANT, value = bg_color)
    elif (64 - n_h)%2 == 0 and (64 - n_w)%2 != 0:
        padded_img = cv2.copyMakeBorder(crop_resize,int((64 - n_h)/2), int((64 - n_h)/2), 
                                    int((64 - n_w)/2), int((64 - n_w)/2) + 1, cv2.BORDER_CONSTANT, value = bg_color)        
    return padded_img

img_assem = np.empty((0, 64, 64), int)
label_assem = np.array([])

for k in range(train_file.shape[0]): #train_file.shape[0]
    label_np = np.array(train_file.labels[k])
    if pd.isnull(label_np).all() == True:
        continue
    label_np = label_np.reshape(int(len(label_np)/5), 5)
    label_list = np.array([])
    for i in label_np[:, 0]:
        label_list= np.append(label_list, unicode_chart.char[unicode_chart.Unicode == i])
    
    image = cv2.imread(os.path.join(r"C:\Users\Kenichi\Documents\Kaggle_Data\kuzushiji_recognition\train_images", 
                                    train_file.image_id[k] + ".jpg"))
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)   
    height, width = img.shape
    
#     fig =plt.figure(figsize=(width/200, height/200))
#     ax = fig.add_subplot(1,1,1)
    j = 0
    
    for i in label_np:
        rect = mpatches.Rectangle((int(i[1]), int(i[2])), int(i[3]), int(i[4]), fill=False, edgecolor='red', linewidth=1)
#         ax.add_patch(rect)
        crop = img[ int(i[2]):int(i[2]) + int(i[4]), int(i[1]):int(i[1]) + int(i[3])]
        crop = get_padded_image(crop)
#         print(crop.shape)
#         cv2.imwrite(os.path.join("../../Kaggle_Data/kuzushiji_recognition/cropped", train_file.image_id[k] + 
#                                  "_" + str(i[0]) + "_" + str(i[1]) + "_" + str(i[2]) + "_"
#                                  + str(i[3]) + "_" + str(i[4]) + ".jpg"), crop)
        img_assem = np.append(img_assem, np.array([crop]), axis = 0)
        label_assem = np.append(label_assem, str(i[0]))
#         ax.text((int(i[1]) + int(i[3]))/width, 1 - int(i[2])/height, label_list[j],
#                 horizontalalignment='left',fontsize=20,
#                 verticalalignment='center',
#                 rotation='horizontal',
#                 transform=ax.transAxes)
        j = j+1
#     ax.imshow(img)
#     plt.savefig(os.path.join("../../Kaggle_Data/kuzushiji_recognition/anotated", train_file.image_id[k]+"_anotated.jpg"))
#plt.show()
    if k%100 == 0:
        print("k:", k)
        np.save('../../Kaggle_Data\kuzushiji_recognition\image_np', img_assem)
        np.save('../../Kaggle_Data\kuzushiji_recognition\label_np', label_assem)
#     if k%500 == 0:
#         img_assem = np.empty((0, 64, 64), int)
#         label_assem = np.array([])

print(img_assem.shape)
print(label_assem.shape)
np.save('../../Kaggle_Data\kuzushiji_recognition\image_np', img_assem)
np.save('../../Kaggle_Data\kuzushiji_recognition\label_np', label_assem)

k: 0
k: 100
k: 200
k: 300
k: 400
k: 500
k: 600
k: 700
k: 800
k: 1000


In [96]:
def(crop):
    h, w, ch = crop.shape
    if w > h:
        crop_resize = cv2.resize(crop , (64, int(h/(w/64))))
    else:
        ratio = int(h/64)
        crop_resize = cv2.resize(crop , (int(w/(h/64)), 64))

    n_h, n_w, ch = crop_resize.shape
    print(int((64 - n_h)/2),int((64 - n_w)/2))
    bg_color = [198, 244, 252]
    if int((64 - n_h)/2)%2 == 0 and int((64 - n_w)/2)%2 == 0:
        padded_img = cv2.copyMakeBorder(crop_resize,int((64 - n_h)/2), int((64 - n_h)/2), 
                                    int((64 - n_w)/2), int((64 - n_w)/2), cv2.BORDER_CONSTANT, value = bg_color)
    elif: int((64 - n_h)/2)%2 != 0 and int((64 - n_w)/2)%2 == 0:
        padded_img = cv2.copyMakeBorder(crop_resize,int((64 - n_h)/2), int((64 - n_h)/2) + 1, 
                                    int((64 - n_w)/2), int((64 - n_w)/2), cv2.BORDER_CONSTANT, value = bg_color)
    elif: int((64 - n_h)/2)%2 == 0 and int((64 - n_w)/2)%2 != 0:
        padded_img = cv2.copyMakeBorder(crop_resize,int((64 - n_h)/2), int((64 - n_h)/2) + 1, 
                                    int((64 - n_w)/2), int((64 - n_w)/2) + 1, cv2.BORDER_CONSTANT, value = bg_color)
            
    return padded_img

0 15


In [145]:
unicode_chart.char[unicode_chart.Unicode == "U+309D"]

128    ゝ
Name: char, dtype: object

In [186]:
np.load('../../Kaggle_Data\kuzushiji_recognition\label_np.npy')

array(['U+306F', 'U+304C', 'U+3044', 'U+3051', 'U+306B', 'U+306B',
       'U+306E', 'U+5DE5', 'U+3053', 'U+4E09', 'U+306E', 'U+3084',
       'U+3068', 'U+5DF1', 'U+3082', 'U+3055', 'U+306E', 'U+306E',
       'U+4E16', 'U+7D30', 'U+305D', 'U+4EBA', 'U+3051', 'U+308C',
       'U+3060', 'U+5F37', 'U+306E', 'U+305F', 'U+3066', 'U+4FF3',
       'U+6839', 'U+304B', 'U+8AE7', 'U+308C', 'U+5B50', 'U+3092',
       'U+53CA', 'U+8005', 'U+305A', 'U+907F', 'U+6B63', 'U+6587',
       'U+3075', 'U+6642', 'U+601D', 'U+306A', 'U+3081', 'U+6D6E',
       'U+3092', 'U+6C17', 'U+8077', 'U+8001', 'U+6B66', 'U+697D',
       'U+3082', 'U+76F2', 'U+82E5', 'U+81EA', 'U+3069', 'U+98A8',
       'U+306B', 'U+88CF', 'U+7573', 'U+606F', 'U+5E8F', 'U+3057',
       'U+3057'], dtype='<U32')

In [154]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Lambda, Input, Dense, Flatten, BatchNormalization
from keras.models import Model
from keras.layers.core import Dropout
from keras import optimizers
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau,TensorBoard

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.preprocessing import OneHotEncoder
from keras.datasets import cifar10
import cv2
# import gc
import numpy as np

print(tf.__version__)
print(keras.__version__)

# Prepare data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train).toarray()
y_test = enc.fit_transform(y_test).toarray()

inputs = Input(shape=(32, 32,3))
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='block1_pool')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='block2_pool')(x)
x = Conv2D(192, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='block3_pool')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='block5_pool')(x)
flattened = Flatten(name='flatten')(x)
x = Dense(1024, activation='relu', name='fc1')(flattened)
x = Dropout(0.5, name='dropout1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = Dropout(0.5, name='dropout2')(x)
predictions = Dense(10, activation='softmax', name='predictions')(x)

BATCH_SIZE = 256
sgd = optimizers.SGD(lr=0.01,
                     momentum=0.9,
                     decay=5e-4)#, nesterov=False)

model = Model(inputs=inputs, outputs=predictions)


model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train
rlop = ReduceLROnPlateau(monitor='val_acc',
                         factor=0.1,
                         patience=5,
                         verbose=1,
                         mode='auto',
                         epsilon=0.0001,
                         cooldown=0,
                         min_lr=0.00001)

 
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=74, verbose=1,
              callbacks=[rlop], validation_data=(x_test, y_test))

y_pred = model.predict(x_test, verbose=1)

1.14.0
2.2.5


C:\Users\Kenichi\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Kenichi\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case yo

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
batch_normalization_10 (Batc (None, 16, 16, 128)       512       
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

C:\Users\Kenichi\Anaconda3\lib\site-packages\keras\callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 50000 samples, validate on 10000 samples
Epoch 1/74
 6144/50000 [==>...........................] - ETA: 6:00 - loss: 2.5207 - acc: 0.2005

KeyboardInterrupt: 